# `HF_classification . ai-forever . sbert_large_mt_nlu_ru` notebook
### Descriprion
This notebook is one of those which were used to training large DL models from `transformers` library and measuring classification performance. Experiments with **sbert-large-mt-nlu-ru** model was conducted here. However, the model was not profoundly explored due to its size and lack of local computational resources.

In [1]:
%cd ../..

/home/pristalovya/Документы/nlp-coursework


In [17]:
from datasets_ import DatasetLoader

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import resample

import matplotlib.pyplot as plt
%matplotlib notebook

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW

from tqdm import tqdm
import numpy as np
import pandas as pd

import sys

from nltk import WhitespaceTokenizer

from IPython.display import clear_output

from transformers import (
    pipeline,                       
    AutoModelForSequenceClassification,                       
    BertForSequenceClassification,                       
    AutoTokenizer,
    AdamW,
)

In [3]:
train, test = DatasetLoader.load_reviews_Review_Label_dataset(train_test_split=True,
                                                              classnames_to_int=True,
                                                              remove_neutral_class=True,
                                                              show_path=True,)
train.label[train['label'] == 2] = 1
test.label[test['label'] == 2] = 1

print(train.shape, test.shape)

/home/pristalovya/Документы/nlp-coursework/data/reviews_Review_Label/reviews_Review_Label.csv
(55346, 2) (23721, 2)


/tmp/ipykernel_7527/3832119039.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.label[train['label'] == 2] = 1
/tmp/ipykernel_7527/3832119039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.label[test['label'] == 2] = 1


In [11]:
train

,review,label
54700,Фильму не стоило бы выходить за пределы Велико...,0
67873,Это один из самых впечатляющих и по-настоящему...,1
44030,Первая часть 'Чужих' в свое время произвела на...,1
21503,История начинающего барабанщика Эндрю зародила...,1
75163,<b>Ну вот мы и дождались. В прокат вышла новая...,1
...,...,...
6265,Всем нам в равной мере свойственно как злорадс...,0
54886,"Даже в таком неординарном фильме. Да, я уверен...",1
76820,"Я не был сильно заинтересован 'Дэдпулом', одна...",1
860,"Фильм, который неизменно занимает первые места...",1


In [13]:
train.label.value_counts()

1    48477
0     6869
Name: label, dtype: int64

In [18]:
train = pd.concat([train, resample(train[train.label == 0], n_samples=41608, random_state=42)])

In [19]:
train

,review,label
54700,Фильму не стоило бы выходить за пределы Велико...,0
67873,Это один из самых впечатляющих и по-настоящему...,1
44030,Первая часть 'Чужих' в свое время произвела на...,1
21503,История начинающего барабанщика Эндрю зародила...,1
75163,<b>Ну вот мы и дождались. В прокат вышла новая...,1
...,...,...
7435,"Есть разведенка, которая совсем не умеет общат...",0
62124,"– Артем, у тебя есть три дня, чтобы предупреди...",0
42486,Что нам обещает показать фильм «Пролетая над г...,0
33186,Это новый мир. Этот фильм раз и навсегда измен...,0


In [20]:
train.label.value_counts()

0    48477
1    48477
Name: label, dtype: int64

In [21]:
test

,review,label
25749,Большое количество фильмов советского кинемато...,1
44489,"Тяжело ответить на вопрос, что же такое Догвил...",1
53162,"В наше время такие героини, как скажем наприме...",0
25843,В 2001 году нам довелось познакомиться с новой...,1
44609,"«Это фильм?», «У них не хватило денег на декор...",1
...,...,...
14104,- Через столько лет?\r\n- Всегда\r\n\r\nБезусл...,1
22232,"После просмотра трейлера, я был под большим вп...",1
73314,"Многие не верят, но я легко подключаюсь к прои...",1
47848,"Как часто нам нужна поддержка? Да, пожалуй, оч...",1


In [4]:
# 1.71 Gb

checkpoint = "ai-forever/sbert_large_mt_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_mt_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_model_input_length=512):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_model_input_length = max_model_input_length
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        review = self.reviews.iloc[idx]
        label = self.labels.iloc[idx]
        review_tokenized = self.tokenizer(
            review,
            add_special_tokens=True,
            max_length=self.max_model_input_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )
        input_ids = review_tokenized['input_ids'].flatten()
        attn_mask = review_tokenized['attention_mask'].flatten()
        
        return {
            'review': review,
            'input_ids': input_ids,
            'attention_mask': attn_mask,
            'label': label,
        }
        

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [6]:
class BertClassifier:
    def __init__(self, checkpoint, n_classes=2):
        
        self.model = BertForSequenceClassification.from_pretrained(checkpoint, ignore_mismatched_sizes=True)
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.max_len = 1024
        self.out_features = 1024
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)
        self.loss_fn = nn.CrossEntropyLoss()
        
        self.optimizer = torch.optim.Adam(self.model.classifier.parameters(), lr=1e-3)
        
        self.all_losses = []
        self.epoch_losses = []
        self.epoch_acc = []       

        
    def fit(self):
        self.model.train()
        losses = []
        correct_predictions = 0
        
        t = tqdm(train_dataloader, file=sys.stdout, ncols=100)

        for data in t:
            input_ids = data['input_ids'].to(self.device)
            attention_mask = data['attention_mask'].to(self.device).to(float)
            labels = data['label'].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = outputs.logits.argmax(dim=1)
            
            loss = self.loss_fn(outputs.logits, labels)

            correct_predictions += torch.sum(preds == labels)

            losses.append(loss.item())

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            t.set_postfix(ordered_dict={'loss': loss.item()}, refresh=True)


        train_acc = correct_predictions.double() / len(train_dataset)
        train_loss = np.mean(losses)
        self.all_losses.extend(losses)
        self.epoch_losses.append(train_loss)
        self.epoch_acc.append(train_acc)
        return train_acc, train_loss
    
    
    def evaluate(self):
        self.model.eval()
        losses = []
        correct_predictions = 0
        
        all_preds = []
        
        t = tqdm(test_dataloader, file=sys.stdout, ncols=100)

        with torch.no_grad():
            for data in t:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                labels = data["label"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, labels)
                correct_predictions += torch.sum(preds == labels)
                
                all_preds.extend(preds.tolist())
                
                losses.append(loss.item())
                
                t.set_postfix(ordered_dict={'loss': loss.item()}, refresh=True)
                
        print('Classification report:')
        print(classification_report(test_dataset.labels, all_preds))

        val_acc = correct_predictions.double() / len(test_dataset)
        val_loss = np.mean(losses)
        return val_acc.item(), val_loss
    

    
    def train(self, n_epochs, pretrain_test=False):
        try:
            best_accuracy = 0

            if pretrain_test:
                print('Pre-training test:')
                val_acc, val_loss = self.evaluate()
                print(f'Test loss {val_loss} accuracy {val_acc}')
                print('-' * 10)

            for epoch in range(n_epochs):
                print(f'Epoch {epoch + 1}/{n_epochs}')
                train_acc, train_loss = self.fit()
                print(f'Train loss {train_loss} accuracy {train_acc}')

                val_acc, val_loss = self.evaluate()
                print(f'Test loss {val_loss} accuracy {val_acc}')
                print('-' * 10)

    
        except KeyboardInterrupt:
            print('Training was manually stopped. ')


In [22]:
clf = BertClassifier("ai-forever/sbert_large_mt_nlu_ru")

train_dataset = ReviewDataset(train.review, train.label, clf.tokenizer)
test_dataset = ReviewDataset(test.review, test.label, clf.tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
for param in clf.model.bert.parameters():
    param.requires_grad = False
print('All parameters:', sum(p.numel() for p in clf.model.parameters()))
print('Trainable parameters:', sum(p.numel() for p in clf.model.parameters() if p.requires_grad))

All parameters: 426910722
Trainable parameters: 2050


In [9]:
clf.train(3)

Epoch 1/3
 32%|██████████████▊                               | 556/1730 [43:54<1:32:43,  4.74s/it, loss=0.381]
Training was manually stopped. 


In [10]:
clf.optimizer = AdamW(clf.model.classifier.parameters(), lr=2e-5, correct_bias=False)
print(clf.optimizer)
clf.train(10, pretrain_test=True)

/home/pristalovya/Inter/linux_packages/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: False
    eps: 1e-06
    lr: 2e-05
    weight_decay: 0.0
)
Pre-training test:
100%|█████████████████████████████████████████████████| 742/742 [56:06<00:00,  4.54s/it, loss=0.127]
Classification report:
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      2979
           1       0.90      0.99      0.94     20742

    accuracy                           0.90     23721
   macro avg       0.83      0.63      0.67     23721
weighted avg       0.89      0.90      0.88     23721

Test loss 0.25290343917484553 accuracy 0.8979385354748957
----------
Epoch 1/10
  4%|█▋                                             | 61/1730 [04:47<2:11:12,  4.72s/it, loss=0.135]
Training was manually stopped. 


In [23]:
clf.optimizer = AdamW(clf.model.classifier.parameters(), lr=2e-5, correct_bias=False)
print(clf.optimizer)
clf.train(10, pretrain_test=False)

/home/pristalovya/Inter/linux_packages/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: False
    eps: 1e-06
    lr: 2e-05
    weight_decay: 0.0
)
Epoch 1/10
100%|█████████████████████████████████████████████| 3030/3030 [3:51:16<00:00,  4.58s/it, loss=0.472]
Train loss 0.4514490670791947 accuracy 0.7965530045176062
100%|█████████████████████████████████████████████████| 742/742 [54:41<00:00,  4.42s/it, loss=0.429]
Classification report:
              precision    recall  f1-score   support

           0       0.38      0.85      0.53      2979
           1       0.97      0.80      0.88     20742

    accuracy                           0.81     23721
   macro avg       0.68      0.83      0.70     23721
weighted avg       0.90      0.81      0.84     23721

Test loss 0.42632369245239343 accuracy 0.8089878167024999
----------
Epoch 2/10
100%|██████████████████████████████████████████████| 3030/3030 [3:49:59<00:00,  4.55s/it, loss=0.46]
Train loss 0.44220961745422666 accuracy 0.8018854302040143
100%|███████